<a href="https://colab.research.google.com/github/bbcx-investments/notebooks/blob/main/options/binomial_trees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

def stockTree(S, u, n):
    return [[S * u ** (t - 2 * i) for i in range(t + 1)] for t in range(n + 1)]

def europeanTree(S, K, r, u, n, kind):
    def f(S):
        if kind == "call":
            return np.maximum(np.array(S) - K, 0)
        else:
            return np.maximum(K - np.array(S), 0)

    d = 1 / u
    p = (1 + r - d) / (u - d)
    disc = 1 / (1 + r)
    ST = [S * u ** (n - 2 * i) for i in range(n + 1)]
    x = f(ST)
    lst = [x]
    while len(x) > 1:
        x = disc * (p * x[:-1] + (1 - p) * x[1:])
        lst.insert(0, x)
    return [list(x) for x in lst], p


def americanTree(S, K, r, u, n, kind):
    def f(S):
        if kind == "call":
            return np.maximum(np.array(S) - K, 0)
        else:
            return np.maximum(K - np.array(S), 0)

    d = 1 / u
    p = (1 + r - d) / (u - d)
    disc = 1 / (1 + r)
    ST = [S * u ** (n - 2 * i) for i in range(n + 1)]
    x = f(ST)
    lst = [x]
    while len(x) > 1:
        x0 = disc * (p * x[:-1] + (1 - p) * x[1:])
        t = len(x0) - 1
        St = [S * u ** (t - 2 * i) for i in range(t + 1)]
        x = np.maximum(x0, f(St))
        lst.insert(0, x)
    return [list(x) for x in lst], p

In [ ]:
# example parameters

S = 50
K = 50
r = 0.02
u = 1.1
n = 3
kind = 'put'

# stock tree

df = pd.DataFrame(stockTree(S, u, n)).transpose()
df.columns = ['period ' + str(i) for i in range(df.shape[0])]
df

,period 0,period 1,period 2,period 3
0,50.0,55.000000,60.500000,66.550000
1,NaN,45.454545,50.000000,55.000000
2,NaN,NaN,41.322314,45.454545
3,NaN,NaN,NaN,37.565740


In [ ]:
# European put tree

df = pd.DataFrame(europeanTree(S, K, r, u, n, kind)[0]).transpose()
df.columns = ['period ' + str(i) for i in range(df.shape[0])]
df

,period 0,period 1,period 2,period 3
0,2.17309,0.767191,0.000000,0.000000
1,NaN,4.225893,1.867414,0.000000
2,NaN,NaN,7.697294,4.545455
3,NaN,NaN,NaN,12.434260


In [ ]:
print('European Put value at date 0:', df.iloc[0,0].round(3))

European Put value at date 0: 2.173


In [ ]:
# American put tree

df = pd.DataFrame(americanTree(S, K, r, u, n, kind)[0]).transpose()
df.columns = ['period ' + str(i) for i in range(df.shape[0])]
df

,period 0,period 1,period 2,period 3
0,2.338563,0.767191,0.000000,0.000000
1,NaN,4.628669,1.867414,0.000000
2,NaN,NaN,8.677686,4.545455
3,NaN,NaN,NaN,12.434260


In [ ]:
print('American Put value at date 0:', df.iloc[0,0].round(3))

American Put value at date 0: 2.339
